In [1]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from fastai.text.all import *
from fastai import *
from blurr import *
#from transformers import *
import transformers
from blurr.text.data.all import *
from blurr.text.modeling.all import *
import bert_score
from bert_score import score

import numpy as np
import pandas as pd
import gzip
import io
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from fastbook import load_learner
import boto3

C:\Users\Yao\Anaconda3\lib\site-packages\blurr\text\modeling\question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Yao\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Yao\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
##!pip install fastbook
##!pip install fastai
#!pip install transformers
##!pip install blurr
#!pip install --upgrade h5py
#!pip install libhdf5
#print(h5py.__version__)
#!pip install bert_score
#!pip install bert-score

In [27]:
#!pip install transformers -Uqq
#!pip install datasets -Uqq
#!pip install bert-score -Uqq
#!pip install sacremoses 
#!pip install ohmeow-blurr -Uqq

In [7]:
df_test = pd.read_csv('dataset-test.csv') # use test file just to load faster
#df_test = df_test[df_test['subreddit'] == 'leagueoflegends'] #subset
test_texts = df_test[['content','summary']]

In [ ]:
#df_train = pd.read_csv('dataset-train.csv')
#train_texts = df_train[['content','summary']]

In [5]:
def pre_processing(data):
    stop_words = set(stopwords.words('english')) #use stop words
    #data = re.sub('(\d+)', '', data) #remove digits
    data = re.sub('\W+',' ', data) #remove special chaacters
    word_tokens = word_tokenize(data.lower()) #tokenize the string
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words] #remove stop words
    #porter_stemmer = PorterStemmer() #stem the words 
    #stemmed_words = [porter_stemmer.stem(word) for word in filtered_sentence]
    wordnet_lemmatizer = WordNetLemmatizer() #lemm the word, does better than stem the word
    lemm_words = [wordnet_lemmatizer.lemmatize(word) for word in filtered_sentence]
    return ' '.join(lemm_words)

In [10]:
def f_rscore(reference, candidate): #assuming both are lists, Rouge
    rscores = []
    rscorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    for i in range(len(reference)):
        rscore = rscorer.score(reference[i], candidate[i])
        rscores.append(list(rscore.values())[0][2]) # fscore
    return rscores

In [11]:
def f_bscore(reference, candidate): #assuming both are lists, Bertscore
    P, R, F1 = score(candidate, reference, lang="en")
    return F1.tolist()

In [8]:
pretrained_model_name = "facebook/bart-large-cnn"
hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name, 
                                                                  model_cls=transformers.BartForConditionalGeneration)

# Create mini-batch and define parameters
hf_batch_tfm = Seq2SeqBatchTokenizeTransform(hf_arch, hf_config, hf_tokenizer, hf_model, 
    task='summarization')

# Simple preprocessing
preprocessor = SummarizationPreprocessor(
    hf_tokenizer,
    text_attr='content',
    target_text_attr='summary',
    max_input_tok_length=256,
    max_target_tok_length=130,
    min_summary_char_length=30,
)

preprocessed_train = preprocessor.process_df(test_texts) #test_texts here


# Prepare data for training
blocks = (Seq2SeqTextBlock(batch_tokenize_tfm=hf_batch_tfm), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader('content'), get_y=ColReader('summary'), splitter=RandomSplitter())
# Batch size can be changed here
dls = dblock.dataloaders(preprocessed_train, bs = 8)

C:\Users\Yao\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


C:\Users\Yao\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


In [9]:
#Define performance metrics
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]}}

#Model
model = BaseModelWrapper(hf_model)
learn_cbs = [BaseModelCallback]
fit_cbs = [Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

#Specify training
learn = Learner(dls, model,
                opt_func=ranger,loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,splitter=partial(blurr_seq2seq_splitter, arch=hf_arch)).to_fp16()

In [39]:
learn.save('reddit_summary.pkl')

Path('models/reddit_summary.pkl.pth')

In [12]:
learner = learn.load('bart_reddit_summary')

In [13]:
#inf_learn = load_learner('models/bart_reddit_summary.pth')

In [14]:
outputs = learn.blurr_generate(test_texts.iloc[0].content, early_stopping=False, num_return_sequences=1, \
                               min_length=30, max_length=50)
for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')

=== Prediction 1 ===
{'generated_texts': 'Racism is, in a weird way, an even touchier topic than rape, even though a single racist act may be less harmful than any sexual assault. There are seething undercurrents of unaddressed issues when'}



$$prediction$$

In [15]:
df_test = pd.read_csv('dataset-test.csv')

In [16]:
df_a = df_test[df_test['subreddit'] == 'AskReddit'].sample(50)[['content','summary']]
df_l = df_test[df_test['subreddit'] == 'leagueoflegends'].sample(50)[['content','summary']]
df_t = df_test[df_test['subreddit'] == 'tifu'].sample(50)[['content','summary']]
df_r = df_test[df_test['subreddit'] == 'relationships'].sample(50)[['content','summary']]
df_ra = df_test[df_test['subreddit'] == 'relationship_advice'].sample(50)[['content','summary']]

In [85]:
#preprocessing optional
df_a['content'] = list(map(pre_processing, df_a['content'])) 
df_a['summary'] = list(map(pre_processing, df_a['summary'])) 
df_l['content'] = list(map(pre_processing, df_l['content'])) 
df_l['summary'] = list(map(pre_processing, df_l['summary'])) 
df_t['content'] = list(map(pre_processing, df_t['content'])) 
df_t['summary'] = list(map(pre_processing, df_t['summary'])) 
df_r['content'] = list(map(pre_processing, df_r['content'])) 
df_r['summary'] = list(map(pre_processing, df_r['summary'])) 
df_ra['content'] = list(map(pre_processing, df_ra['content'])) 
df_ra['summary'] = list(map(pre_processing, df_ra['summary'])) 

In [17]:
df_a.iloc[0].content

"Putting an astronaut into orbit is expensive (currently about $65 million) so the space agency that bought the ticket obviously wants to get its money's worth out of their employee. \n Most astronaut work is maintaining science projects onboard the station. Some astronauts are trained in EVA maintenance and repair, so they have to do things like fix broken ammonia coolant pumps (like we saw last week) or service pallets of exposed materials outside the station, or inspect areas of the ISS that can't be viewed from the onboard cameras. \n The International Space Station is fueled by money, so most days, the astronauts conduct press conferences with Earthbound reporters. The public relations aspect is critical for maintaining support for spending tax dollars, so it's why astronauts talk to the assorted Hot 96s and Kiss 105s and The Q-101s around the country - - taxpayers need to remember there are people up there in space. \n In order to reduce osteoporosis and muscle atrophy, astronaut

In [18]:
def make_predictions(test_texts):
    #list(outputs[0].values())
    predictions = []
    for i in range(len(test_texts)):
        outputs = learn.blurr_generate(test_texts.iloc[i].content, early_stopping=False, num_return_sequences=1, \
                                   min_length=30, max_length=50)
        predictions.append(list(outputs[0].values())[0])
    return predictions

In [19]:
a_p = make_predictions(df_a)
l_p = make_predictions(df_l)
t_p = make_predictions(df_t)
r_p = make_predictions(df_r)
ra_p = make_predictions(df_ra)

In [20]:
def avg_rouge(test_texts, predictions):
    score_list = f_rscore(list(test_texts.summary), predictions)
    return np.average(score_list)

In [21]:
def avg_bertscore(test_texts, predictions):
    score_list = f_bscore(list(test_texts.summary), predictions)
    return np.average(score_list)

In [22]:
avg_rouge(df_a, a_p)

0.1409546271588405

In [23]:
avg_rouge(df_l, l_p)

0.1390674058598556

In [24]:
avg_rouge(df_t, t_p)

0.14295083612270246

In [25]:
avg_rouge(df_r, r_p)

0.15826219750022819

In [26]:
avg_rouge(df_ra, ra_p)

0.16458580163167766

In [27]:
avg_bertscore(df_a, a_p)

0.8587509858608245

In [28]:
avg_bertscore(df_l, l_p)

0.8485599493980408

In [30]:
avg_bertscore(df_t, t_p)

0.8507759821414947

In [29]:
avg_bertscore(df_r, r_p)

0.8596891975402832

In [31]:
avg_bertscore(df_ra, ra_p)

0.8590439307689667

In [74]:
score_list = f_rscore(list(df_t.summary), t_p)
[(i ,e) for i, e in enumerate(score_list) if e < 0.05]

[(3, 0.038461538461538464),
 (10, 0.0),
 (23, 0.049999999999999996),
 (27, 0.046511627906976744)]

In [75]:
score_list = f_rscore(list(df_t.summary), t_p)
[(i ,e) for i, e in enumerate(score_list) if e >0.2]

[(0, 0.21875),
 (1, 0.2903225806451613),
 (2, 0.339622641509434),
 (11, 0.2153846153846154),
 (21, 0.24324324324324323),
 (25, 0.2318840579710145),
 (26, 0.24000000000000005),
 (28, 0.3013698630136986),
 (33, 0.25396825396825395),
 (40, 0.24615384615384614),
 (42, 0.22641509433962265)]

$$Example$$

In [86]:
df_t.iloc[10].content

'This happened today around 12am. \n I had just left my girlfriend\'s house and we said our goodbyes, then entered my car. I\'m just thinking what a great time it would be to have a cigarette, but I wait a bit before I light it, so I start the car and all of a sudden a SPIDER the size of megalodon crawls onto my front windshield, so my first reaction is to freeze and assess the situation; is it inside or outside? I am extremely afraid of spiders so you can imagine what it\'d be like. After waiting for confirmation that it\'s outside, I used the wipers to push it off and it runs for it\'s dear life. I\'m thinking "haha, gotcha", then it\'s gone. No sign of monster spider. I roll my windows down and light my cigarette when all of a sudden, it\'s back. The spider crawls back onto the front windshield, just sitting there. I continuously shit myself and regret getting into my car, I close the windows and at the same time I panic because the cigarette smell will definitely stain more without

In [87]:
df_t.iloc[10].summary

'nature is scary'

In [88]:
t_p[10]

"A spider the size of a megalodon crawls onto my windshield. I am extremely afraid of spiders so you can imagine what it'd be like to have one crawl on you. I continued to be paranoid for the rest"

$$Example$$

In [92]:
df_t.iloc[1].content

'Hello TIFU. I knew one day I would have a story thats TIFU worthy. I just didnt expect it to come so early. \n and this actually is happening today. \n Im a 22 yo undergrad student, majoring in Mass communication with a emphasis on Public Relations and Advertising. Im pretty well into my degree, past my core classes and actually into the classes that relate to my major. One of the classes is an Advertising analysis and study class. Basically, we learn about what makes a good advertisement, what kind of response we should get, and how to interest an audience im our product. \n One of the major projects in this class is a short presentation about a couple of advertisements, one good, one bad, what kind of response they provoke and how to make them better. We then have to present them in front of the class, in the form of a short powerpoint. Seems simple enough, right? So, yesterday, I got up and went to a different class, knowing that at some point I would need to review my presentation

In [93]:
df_t.iloc[1].summary

'Burned the mightnight oil a little too hard. Forgot to set my alarm to wake up, missed presentation day for my project. \n TIFU'

In [94]:
t_p[1]

' Im a 22-year-old student majoring in Mass communication with a emphasis on Public Relations and Advertising. I got distracted by the internet and forgot to set my alarm for my presentation day, so I missed my presentation.'

$$Example$$

In [129]:
df_a.iloc[24].content

'My best friend teaches freshman english. After class one day, a note was left and a group of girls in the next class grabbed it, read it, and started giggling. My friend grabbed the note from the girls, and threw it away without reading it, trying to save the student (a quite and polite student), and probably herself, some humiliation. However, when the next day rolled around and the same girl left another note behind, it was fair game. The 14-year-old had left behind a note addressed to her boyfriend and it read: "Hey, wanna come over after school for some sex and capri sun?". Unsure of what Capri Sun had to do with sex, my friend decided to search for it on urbandictionary.com. "Capri Sun" means getting jizzed on the face after a blow job. She was never able to look at that girl the same.'

In [130]:
df_a.iloc[24].summary

'14-year old freshman wanted her boyfriend to come on her face.'

In [131]:
a_p[24]

'"Capri Sun" means getting jizzed on the face after a blow job. The 14-year-old had left behind a note addressed to her boyfriend and it read: "Hey, wanna come over after school for some'

$$Real Example$$

In [32]:
content = ["Hello David, my name is George and I'm currently a MIDS candidate at UC Berkeley.I'm a big fan of your work and my team of data scientists and developers have created a product I think you’d be interested in. Do you know the hardest part about coming up with an elevator pitch? It’s trying to condense the most important parts into a concise number of words. That same difficulty can be found on Reddit, one of the fastest growing social media sites in the world. We’ve analyzed reddit data over a 10-year period and found that nearly 20% of all posts and articles had ineffective or erroneous titles. So, what’s the solution? Our product is a web-based app that takes a user’s post and generates a title that has been evaluated to accurately summarize the content. We're still in the early stages of development and would appreciate any feedback, would you be interested in meeting with my team to talk more through our product?"]
outputs = learn.blurr_generate(content, early_stopping=False, num_return_sequences=1, \
                               min_length=30, max_length=50)
for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')

=== Prediction 1 ===
{'generated_texts': 'A UC Berkeley student has created an app that generates a title for a user’s post that can accurately summarize the content of a post. The product is still in the early stages of development and would appreciate any feedback.'}



$$BertScore Example$$

In [96]:
score_list = f_bscore(list(df_t.summary), t_p)
[(i ,e) for i, e in enumerate(score_list) if e < 1]

[(0, 0.8793852925300598),
 (1, 0.8709421753883362),
 (2, 0.8801113963127136),
 (3, 0.8368353247642517),
 (4, 0.8676187992095947),
 (5, 0.8566980361938477),
 (6, 0.8403998017311096),
 (7, 0.8571620583534241),
 (8, 0.8545660972595215),
 (9, 0.8320644497871399),
 (10, 0.8259784579277039),
 (11, 0.879986047744751),
 (12, 0.8258947134017944),
 (13, 0.8445872068405151),
 (14, 0.8670421838760376),
 (15, 0.8418312668800354),
 (16, 0.8355985879898071),
 (17, 0.8534197807312012),
 (18, 0.847183108329773),
 (19, 0.7866148948669434),
 (20, 0.8464949131011963),
 (21, 0.8735225796699524),
 (22, 0.8166868686676025),
 (23, 0.8529530763626099),
 (24, 0.8596628308296204),
 (25, 0.8414990305900574),
 (26, 0.885001540184021),
 (27, 0.8319551944732666),
 (28, 0.8730170726776123),
 (29, 0.8451743125915527),
 (30, 0.8385426998138428),
 (31, 0.8534262180328369),
 (32, 0.8014107942581177),
 (33, 0.8693335652351379),
 (34, 0.8324257135391235),
 (35, 0.8394963145256042),
 (36, 0.8568946123123169),
 (37, 0.846091